In [3]:
# import dependencies and API key
import pandas as pd
import gmaps
import requests
from config import g_key

In [4]:
file_to_load = 'weather_data/cities.csv'
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kuching,MY,2021-09-11 01:32:57,1.5500,110.3333,80.82,83,20,6.91
1,1,Sattahip,TH,2021-09-11 01:29:08,12.6664,100.9007,82.49,78,40,5.75
2,2,Charters Towers,AU,2021-09-11 01:32:58,-20.1000,146.2667,77.05,49,75,16.49
3,3,Bluff,NZ,2021-09-11 01:32:58,-46.6000,168.3333,44.80,68,90,19.80
4,4,Ushuaia,AR,2021-09-11 01:30:44,-54.8000,-68.3000,37.06,81,20,0.00


In [5]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
gmaps.configure(api_key=g_key)

In [25]:
# Heatmap of temperature
# Get lat and lng
locations = city_data_df[['Lat', 'Lng']]
# Get max temp
temperatures = city_data_df['Max Temp']
# Create the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0, layout={'height': '700px', 'width': '1100px'})
# Assign the heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in city_data_df['Max Temp']], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='700px', width='1100px'))

In [26]:
# Heatmap of percent humidity
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0, layout={'height': '700px', 'width': '1100px'})
# Assign the heatmap layer
heatmap_layer = gmaps.heatmap_layer(city_data_df[['Lat', 'Lng']], weights=city_data_df['Humidity'], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='700px', width='1100px'))

In [27]:
# Heatmap of percent cloudiness
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0, layout={'height': '700px', 'width': '1100px'})
# Assign the heatmap layer
heatmap_layer = gmaps.heatmap_layer(city_data_df[['Lat', 'Lng']], weights=city_data_df['Cloudiness'], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='700px', width='1100px'))

In [28]:
# Heatmap of Wind Speed
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0, layout={'height': '700px', 'width': '1100px'})
# Assign the heatmap layer
heatmap_layer = gmaps.heatmap_layer(city_data_df[['Lat', 'Lng']], weights=city_data_df['Wind Speed'], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='700px', width='1100px'))

In [29]:
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

In [31]:
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kuching,MY,2021-09-11 01:32:57,1.5500,110.3333,80.82,83,20,6.91
1,1,Sattahip,TH,2021-09-11 01:29:08,12.6664,100.9007,82.49,78,40,5.75
2,2,Charters Towers,AU,2021-09-11 01:32:58,-20.1000,146.2667,77.05,49,75,16.49
6,6,Kapaa,US,2021-09-11 01:32:59,22.0752,-159.3190,84.52,68,20,5.01
7,7,Vaini,TO,2021-09-11 01:32:59,-21.2000,-175.2000,78.96,78,20,6.91


In [32]:
preferred_cities_df.count()

City_ID       175
City          175
Country       175
Date          175
Lat           175
Lng           175
Max Temp      175
Humidity      175
Cloudiness    175
Wind Speed    175
dtype: int64

In [33]:
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng',]].copy()
hotel_df['Hotel'] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel
0,Kuching,MY,80.82,1.5500,110.3333,
1,Sattahip,TH,82.49,12.6664,100.9007,
2,Charters Towers,AU,77.05,-20.1000,146.2667,
6,Kapaa,US,84.52,22.0752,-159.3190,
7,Vaini,TO,78.96,-21.2000,-175.2000,


In [36]:
import requests
# Set params to search for a hotel
params = {
    'radius': 5000,
    'key': g_key,
    'type': 'lodging'
}
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    params['location'] = f'{lat},{lng}'

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    try:
        hotels = requests.get(base_url, params=params).json()
        hotel_df.loc[index, 'Hotel'] = hotels['results'][0]['name']
    except:
        print('Hotel not found. Skipping...')
        continue

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [37]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel
0,Kuching,MY,80.82,1.5500,110.3333,The Ranee Boutique Suites
1,Sattahip,TH,82.49,12.6664,100.9007,Pob Choke Residences ( พบโชคเรดซิเด้นท์ )
2,Charters Towers,AU,77.05,-20.1000,146.2667,Cattleman's Rest Motor Inn
6,Kapaa,US,84.52,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Vaini,TO,78.96,-21.2000,-175.2000,Keleti Beach Resort
8,Yantai,CN,78.62,37.5333,121.4000,Yantai Seaside International Youth Hostel
9,Port Macquarie,AU,78.76,-31.4333,152.9167,Rydges Port Macquarie
17,Karratha,AU,86.00,-20.7377,116.8463,ibis Styles Karratha
20,Mackay,AU,77.04,-21.1500,149.2000,International Lodge Motel
21,Avarua,CK,80.65,-21.2078,-159.7750,Paradise Inn


In [41]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [43]:
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
fig = gmaps.figure(center=(20.0, 31.0), zoom_level=2.0, layout={'height': '700px', 'width': '1100px'})
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='700px', width='1100px'))